# Nanopublication Signer & Publisher

Signs and publishes nanopublication `.trig` files to the Nanopub network via Nanodash.

---

## Prerequisites

1. **Install the nanopub library**: `pip install nanopub`
2. **Set up your profile**: Run `np setup` in terminal to create your ORCID-linked profile
3. **Have `.trig` files ready**: Generated from the nanopub generator notebooks

---

## Instructions

1. **Set input path(s)** in Section 1 (single file or directory)
2. **Configure options** (test server, auto-publish)
3. **Run All Cells** → Get signed & published nanopubs

---
# 📁 SECTION 1: INPUT CONFIGURATION (EDIT THIS)
---

In [ ]:
# ============================================
# OPTION A: Single .trig file
# ============================================
INPUT_FILE = "../output/aida/aida_qomic_01.trig"

# ============================================
# OPTION B: Directory of .trig files (processes all)
# ============================================
# INPUT_FILE = "../output/aida/"  # All .trig files in directory

# ============================================
# OPTION C: List of specific files
# ============================================
# INPUT_FILE = [
#     "../output/aida/aida_qomic_01.trig",
#     "../output/software/software_qomic.trig",
#     "../output/cito/cito_qomic_trrust.trig",
# ]

In [ ]:
# ============================================
# PUBLISHING OPTIONS
# ============================================

# Set to True to actually publish (False = sign only)
PUBLISH = True

# Set to True to use test server (for testing, won't be on main network)
USE_TEST_SERVER = False

# Output directory for signed files (None = same as input)
OUTPUT_DIR = None  # e.g., "../output/signed/"

# Skip files that have already been signed (check for .signed.trig)
SKIP_ALREADY_SIGNED = True

---
# ⚙️ SECTION 2: SETUP
---

In [ ]:
# Install dependencies (uncomment if needed)
# !pip install nanopub rdflib

In [ ]:
from pathlib import Path
from datetime import datetime
import json

print("Loading nanopub library...")
try:
    from nanopub import Nanopub, NanopubConf, load_profile
    print("✓ nanopub library loaded")
except ImportError:
    print("❌ nanopub library not found")
    print("   Install with: pip install nanopub")
    raise

print("\n✓ Setup complete")

---
# 👤 SECTION 3: LOAD PROFILE
---

In [ ]:
# Load your nanopub profile
print("Loading nanopub profile...")
try:
    profile = load_profile()
    print(f"✓ Profile loaded")
    print(f"  Name:  {profile.name}")
    print(f"  ORCID: {profile.orcid_id}")
except Exception as e:
    print(f"❌ Failed to load profile: {e}")
    print("\nTo create a profile, run in terminal:")
    print("  np setup")
    raise

In [ ]:
# Create configuration
conf = NanopubConf(
    profile=profile,
    use_test_server=USE_TEST_SERVER
)

server_type = "TEST" if USE_TEST_SERVER else "PRODUCTION"
print(f"✓ Configuration ready (Server: {server_type})")

---
# 📂 SECTION 4: COLLECT INPUT FILES
---

In [ ]:
def collect_trig_files(input_spec):
    """
    Collect .trig files from various input specifications.
    
    Args:
        input_spec: Can be:
            - str: Path to single file or directory
            - list: List of file paths
            - Path: Path object
    
    Returns:
        List of Path objects to .trig files
    """
    files = []
    
    if isinstance(input_spec, list):
        # List of files
        for f in input_spec:
            p = Path(f)
            if p.exists() and p.suffix == '.trig':
                files.append(p)
            else:
                print(f"⚠ Skipping (not found or not .trig): {f}")
    else:
        path = Path(input_spec)
        if path.is_file():
            # Single file
            if path.suffix == '.trig':
                files.append(path)
            else:
                print(f"⚠ Not a .trig file: {path}")
        elif path.is_dir():
            # Directory - find all .trig files
            files = list(path.glob("*.trig"))
            # Exclude already signed files
            files = [f for f in files if not f.stem.endswith('.signed')]
        else:
            print(f"❌ Path not found: {path}")
    
    return sorted(files)


# Collect files
trig_files = collect_trig_files(INPUT_FILE)

if not trig_files:
    print("❌ No .trig files found!")
    print(f"   Input: {INPUT_FILE}")
else:
    print(f"✓ Found {len(trig_files)} .trig file(s):")
    for f in trig_files:
        print(f"   • {f}")

In [ ]:
# Filter out already signed files if requested
if SKIP_ALREADY_SIGNED:
    original_count = len(trig_files)
    
    def has_signed_version(f):
        signed_path = f.parent / f"{f.stem}.signed.trig"
        return signed_path.exists()
    
    trig_files = [f for f in trig_files if not has_signed_version(f)]
    
    skipped = original_count - len(trig_files)
    if skipped > 0:
        print(f"ℹ Skipped {skipped} already-signed file(s)")

print(f"\n📋 Files to process: {len(trig_files)}")

---
# 🔏 SECTION 5: SIGN & PUBLISH
---

In [ ]:
def sign_and_publish(trig_path, conf, publish=True, output_dir=None):
    """
    Sign and optionally publish a single nanopublication.
    
    Args:
        trig_path: Path to the .trig file
        conf: NanopubConf configuration
        publish: Whether to publish after signing
        output_dir: Directory for signed file (None = same as input)
    
    Returns:
        dict with results
    """
    result = {
        'input': str(trig_path),
        'signed_path': None,
        'published_uri': None,
        'success': False,
        'error': None
    }
    
    try:
        # Load the nanopub
        np_obj = Nanopub(rdf=trig_path, conf=conf)
        
        # Sign it
        np_obj.sign()
        
        # Determine output path
        if output_dir:
            out_dir = Path(output_dir)
            out_dir.mkdir(parents=True, exist_ok=True)
            signed_path = out_dir / f"{trig_path.stem}.signed.trig"
        else:
            signed_path = trig_path.parent / f"{trig_path.stem}.signed.trig"
        
        # Save signed version
        np_obj.store(signed_path)
        result['signed_path'] = str(signed_path)
        
        # Publish if requested
        if publish:
            np_obj.publish()
            result['published_uri'] = np_obj.source_uri
        
        result['success'] = True
        
    except Exception as e:
        result['error'] = str(e)
    
    return result

In [ ]:
# Process all files
results = []
success_count = 0
fail_count = 0

print("=" * 70)
print(f"PROCESSING {len(trig_files)} NANOPUBLICATION(S)")
print(f"Mode: {'SIGN + PUBLISH' if PUBLISH else 'SIGN ONLY'}")
print(f"Server: {'TEST' if USE_TEST_SERVER else 'PRODUCTION'}")
print("=" * 70)
print()

for i, trig_file in enumerate(trig_files, 1):
    print(f"[{i}/{len(trig_files)}] Processing: {trig_file.name}")
    
    result = sign_and_publish(
        trig_path=trig_file,
        conf=conf,
        publish=PUBLISH,
        output_dir=OUTPUT_DIR
    )
    results.append(result)
    
    if result['success']:
        success_count += 1
        print(f"    ✓ Signed: {result['signed_path']}")
        if result['published_uri']:
            print(f"    ✓ Published: {result['published_uri']}")
    else:
        fail_count += 1
        print(f"    ❌ Error: {result['error']}")
    print()

print("=" * 70)
print(f"COMPLETE: {success_count} succeeded, {fail_count} failed")
print("=" * 70)

---
# 📊 SECTION 6: RESULTS SUMMARY
---

In [ ]:
# Display results table
print("\n" + "=" * 70)
print("RESULTS SUMMARY")
print("=" * 70)
print(f"{'File':<40} {'Status':<10} {'URI/Error'}")
print("-" * 70)

for r in results:
    filename = Path(r['input']).name[:38]
    if r['success']:
        status = "✓ OK"
        detail = r['published_uri'] or "(signed only)"
    else:
        status = "❌ FAIL"
        detail = r['error'][:40] if r['error'] else "Unknown error"
    
    print(f"{filename:<40} {status:<10} {detail}")

print("=" * 70)

In [ ]:
# Export published URIs
published_uris = [
    {
        'file': Path(r['input']).name,
        'uri': r['published_uri']
    }
    for r in results
    if r['success'] and r['published_uri']
]

if published_uris:
    print("\n📋 PUBLISHED NANOPUBLICATION URIs:")
    print("-" * 70)
    for item in published_uris:
        print(f"• {item['file']}")
        print(f"  {item['uri']}")
    
    # Save to JSON
    output_json = Path(OUTPUT_DIR or ".") / "published_nanopubs.json"
    with open(output_json, 'w') as f:
        json.dump({
            'published_at': datetime.now().isoformat(),
            'server': 'test' if USE_TEST_SERVER else 'production',
            'publisher': profile.name,
            'orcid': profile.orcid_id,
            'nanopublications': published_uris
        }, f, indent=2)
    print(f"\n✓ URIs saved to: {output_json}")

---
# 🔍 SECTION 7: VERIFY PUBLICATION (OPTIONAL)
---

In [ ]:
# Verify a published nanopub by fetching it
VERIFY_URI = None  # Set to a published URI to verify, e.g.:
# VERIFY_URI = "https://w3id.org/np/RAxxxxxxxxx"

if VERIFY_URI:
    print(f"Verifying: {VERIFY_URI}")
    try:
        from nanopub import Nanopub
        fetched = Nanopub(source_uri=VERIFY_URI)
        print("✓ Nanopub successfully retrieved!")
        print(f"  Label: {fetched.rdf.value(fetched.assertion, RDFS.label)}")
    except Exception as e:
        print(f"❌ Verification failed: {e}")
elif published_uris:
    # Auto-verify first published nanopub
    first_uri = published_uris[0]['uri']
    print(f"Auto-verifying first publication: {first_uri}")
    try:
        from nanopub import Nanopub
        fetched = Nanopub(source_uri=first_uri)
        print("✓ Nanopub successfully retrieved from network!")
    except Exception as e:
        print(f"⚠ Could not verify (may need time to propagate): {e}")
else:
    print("ℹ No nanopubs to verify")

---
# 📖 USAGE GUIDE

## Quick Start

```python
# Single file
INPUT_FILE = "path/to/nanopub.trig"

# All files in directory
INPUT_FILE = "path/to/output/aida/"

# Specific files
INPUT_FILE = [
    "output/aida/aida_01.trig",
    "output/software/software_01.trig"
]
```

## Options

| Option | Default | Description |
|--------|---------|-------------|
| `PUBLISH` | `True` | Publish after signing |
| `USE_TEST_SERVER` | `False` | Use test network |
| `OUTPUT_DIR` | `None` | Custom output directory |
| `SKIP_ALREADY_SIGNED` | `True` | Skip files with existing .signed.trig |

## Profile Setup

If you haven't set up a nanopub profile:

```bash
# In terminal
np setup
```

This will:
1. Create RSA keys for signing
2. Link to your ORCID
3. Store profile in `~/.nanopub/profile.yml`

## Batch Publishing Workflow

1. Generate nanopubs with the generator notebooks
2. Set `INPUT_FILE = "../output/"` to process all subdirectories
3. Run this notebook to sign & publish all at once

---